In [1]:
import pandas as pd
import numpy as np
import sklearn
from sklearn import linear_model
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt


In [102]:
# import something
master_df = pd.read_csv('../datasets/MASTER_DF.csv')
master_df.columns

Index(['entry', 'product', 'reacts', 'PubChem', 'SMILES', 'Mol', 'Fingerprint',
       'dist', 'enzyme_class_1', 'enzyme_class_2', 'enzyme_class_3',
       'enzyme_class_4', 'enzyme_class_5', 'enzyme_class_6', 'enzyme_class_7',
       'n_C', 'n_H', 'n_O', 'n_N', 'n_P', 'n_S', 'n_X', 'DoU', 'MW'],
      dtype='object')

In [116]:
feature_df = master_df[['PubChem', 'dist', 'enzyme_class_1', 'enzyme_class_2', 'enzyme_class_3',
       'enzyme_class_4', 'enzyme_class_5', 'enzyme_class_6', 'enzyme_class_7',
        'n_O', 'n_N', 'n_P', 'n_S', 'n_X', 'DoU']]
feature_df.set_index(keys=['PubChem'], inplace=True)
feature_df.head()

,dist,enzyme_class_1,enzyme_class_2,enzyme_class_3,enzyme_class_4,enzyme_class_5,enzyme_class_6,enzyme_class_7,n_O,n_N,n_P,n_S,n_X,DoU
PubChem,,,,,,,,,,,,,,
3394,0.000000,1,0,0,0,0,0,0,3.0,0.0,0.0,1.0,0.0,0.0
3394,0.511007,1,0,0,0,0,0,0,3.0,0.0,0.0,1.0,0.0,0.0
3578,0.000000,1,0,0,0,0,0,0,0.0,0.0,0.0,1.0,0.0,0.0
3578,0.241667,0,1,0,0,0,0,0,0.0,0.0,0.0,1.0,0.0,0.0
3578,0.294605,0,0,0,1,0,0,0,0.0,0.0,0.0,1.0,0.0,0.0


In [117]:
features = np.array(feature_df) #shape balance array for regression
reactions = list(master_df['reacts'])

feature_train, feature_test, reaction_train, reaction_test = train_test_split(features, reactions,
                                                    test_size=0.20, random_state=42)

In [118]:
model_1 = linear_model.LogisticRegression(solver='liblinear', penalty='l1', random_state=1, class_weight='balanced')
model_1.fit(feature_train, np.ravel(reaction_train))

predictions = model_1.predict(feature_test) # change me to the data you want to predict based on 

score = model_1.score(feature_test, reaction_test)
decision = model_1.decision_function(feature_test)
params = model_1.get_params()
pred_log = model_1.predict_log_proba(feature_test)

pred = model_1.predict_proba(feature_test)

score, pred, decision, model_1.classes_, model_1.coef_

(0.9200581395348837, array([[0.97567184, 0.02432816],
        [0.58845931, 0.41154069],
        [0.91923651, 0.08076349],
        ...,
        [0.68742977, 0.31257023],
        [0.75808899, 0.24191101],
        [0.98838878, 0.01161122]]), array([-3.69149187, -0.35759988, -2.43201839, ..., -0.78813049,
        -1.14223086, -4.44410399]), array([0., 1.]), array([[ 1.97960417e+01,  0.00000000e+00, -1.00228061e-01,
         -3.21945182e-01,  9.67342967e-01, -8.21030905e-01,
          7.00151867e-02,  0.00000000e+00, -6.84202661e-02,
          7.34978590e-02, -1.88944112e-02,  2.06923333e-01,
          6.01807845e-01, -1.16681085e-01]]))

In [119]:
prediction_values = pd.DataFrame(pred)
model_descriptive_df = pd.DataFrame()
model_descriptive_df['0']=prediction_values[0]
model_descriptive_df['1']=prediction_values[1]
model_descriptive_df

,0,1
0,0.975672,0.024328
1,0.588459,0.411541
2,0.919237,0.080763
3,0.993312,0.006688
4,0.964063,0.035937
5,0.909015,0.090985
6,0.845677,0.154323
7,0.640765,0.359235
8,0.982030,0.017970
9,0.798240,0.201760


In [120]:
updated = predictions.tolist()
confusion_matrix = confusion_matrix(reaction_test, updated)
print(confusion_matrix)
# upper left and lower right are correct: 1427
# lower left and upper right are incorrect : 99 

TypeError: 'numpy.ndarray' object is not callable

In [121]:
print(classification_report(reaction_test, predictions))

              precision    recall  f1-score   support

         0.0       0.95      0.95      0.95      1064
         1.0       0.82      0.83      0.82       312

   micro avg       0.92      0.92      0.92      1376
   macro avg       0.88      0.89      0.89      1376
weighted avg       0.92      0.92      0.92      1376



In [14]:
model_2 = linear_model.LogisticRegressionCV(solver='liblinear', penalty='l1', random_state=1, cv=10)
model_2.fit(feature_train, np.ravel(reaction_train))
predictions2 = model_2.predict(feature_test)
score2 = model_2.score(feature_test, reaction_test)
decision2 = model_2.decision_function(feature_test)
params2 = model_2.get_params()
pred_log2 = model_2.predict_log_proba(feature_test)
pred2 = model_2.predict_proba(feature_test)

score2, pred2, #decision2, model_2.classes_, model_2.coef_

(0.9436435124508519, array([[0.93313034, 0.06686966],
        [0.71411113, 0.28588887],
        [0.98480214, 0.01519786],
        ...,
        [0.00532237, 0.99467763],
        [0.75659365, 0.24340635],
        [0.87789801, 0.12210199]]))

plan: ridge regression, drop off the least important features and get the AUC ROC curve-> store, drop the last
RFE again? (it didn't show anything...)


______________
# linear regression on default data

logreg1=linear_model.LogisticRegression(random_state=1)        
logreg1.fit(feature_train, np.ravel(reaction_train)) #fit linear model, and shape default array for regression
score = logreg1.score(feature_test, reaction_test)
decision = logreg1.decision_function(feature_test)
params = logreg1.get_params()
pred_log = logreg1.predict_log_proba(feature_test)
pred = logreg1.predict_proba(feature_test)

score, pred, logreg1.classes_, logreg1.coef_ #np.ndarray.shape(pred)

#print('B0, B1: ',logreg.intercept_, logreg.coef_[0])

predictions1 = logreg1.predict(feature_test)
confusion_matrix = confusion_matrix(reaction_test, predictions1)
print(confusion_matrix)
# upper left and lower right are correct: 1400 for the first go
# lower left and upper right are incorrect : 111 for the first go

print(classification_report(reaction_test, predictions1))

logit_roc_auc = roc_auc_score(reaction_test, logreg1.predict(feature_test))
fpr, tpr, thresholds = roc_curve(reaction_test, logreg1.predict_proba(feature_test)[:,1])
plt.figure()
plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % logit_roc_auc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.savefig('Log_ROC')
plt.show()

plt.scatter(reaction_test, predictions1, alpha=0.1)
plt.xlabel('True value')
plt.ylabel('Predicted value')

logreg5 = linear_model.LogisticRegression(penalty='l1', random_state=1)
logreg5.fit(feature_train, np.ravel(reaction_train))
predictions5 = logreg5.predict(feature_test)
score = logreg5.score(feature_test, reaction_test)
decision = logreg5.decision_function(feature_test)
params = logreg5.get_params()
pred_log = logreg5.predict_log_proba(feature_test)
pred = logreg5.predict_proba(feature_test)

score, pred, logreg5.classes_, logreg5.coef_

print(classification_report(reaction_test, predictions5))

logit_roc_auc = roc_auc_score(reaction_test, logreg5.predict(feature_test))
fpr, tpr, thresholds = roc_curve(reaction_test, logreg5.predict_proba(feature_test)[:,1])
plt.figure()
plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % logit_roc_auc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.savefig('Log_ROC')
plt.show()

In [16]:
from sklearn.feature_selection import RFE



In [122]:
rfe_results = pd.DataFrame(columns=['score']) 

rank_array = np.zeros((14,15))

for i in range(1,14):
    logreg2=linear_model.LogisticRegression(solver='liblinear', penalty='l1', random_state=1, max_iter=1000, class_weight='balanced') 

    rfe = RFE(logreg2, i)
    rfe = rfe.fit(feature_train, np.ravel(reaction_train))
    score = rfe.score(feature_test, reaction_test)

    rfe_results.loc[i, 'score'] = score
    for j in range(0,14):  
        rank_array[i, j] = rfe.ranking_[j]

In [123]:
rank_df = pd.DataFrame(rank_array, columns=['dist', 'enzyme_class_1', 'enzyme_class_2', 'enzyme_class_3',
       'enzyme_class_4', 'enzyme_class_5', 'enzyme_class_6', 'enzyme_class_7',
       'n_O', 'n_N', 'n_P', 'n_S', 'n_X', 'DoU', 'drop'])
#rfe_results
rank_df.drop(rank_df.index[0], inplace=True)
rank_df.drop(['drop'], axis=1, inplace=True)
rank_df

,dist,enzyme_class_1,enzyme_class_2,enzyme_class_3,enzyme_class_4,enzyme_class_5,enzyme_class_6,enzyme_class_7,n_O,n_N,n_P,n_S,n_X,DoU
1,1.0,14.0,6.0,5.0,2.0,3.0,11.0,13.0,10.0,9.0,12.0,7.0,4.0,8.0
2,1.0,13.0,5.0,4.0,1.0,2.0,10.0,12.0,9.0,8.0,11.0,6.0,3.0,7.0
3,1.0,12.0,4.0,3.0,1.0,1.0,9.0,11.0,8.0,7.0,10.0,5.0,2.0,6.0
4,1.0,11.0,3.0,2.0,1.0,1.0,8.0,10.0,7.0,6.0,9.0,4.0,1.0,5.0
5,1.0,10.0,2.0,1.0,1.0,1.0,7.0,9.0,6.0,5.0,8.0,3.0,1.0,4.0
6,1.0,9.0,1.0,1.0,1.0,1.0,6.0,8.0,5.0,4.0,7.0,2.0,1.0,3.0
7,1.0,8.0,1.0,1.0,1.0,1.0,5.0,7.0,4.0,3.0,6.0,1.0,1.0,2.0
8,1.0,7.0,1.0,1.0,1.0,1.0,4.0,6.0,3.0,2.0,5.0,1.0,1.0,1.0
9,1.0,6.0,1.0,1.0,1.0,1.0,3.0,5.0,2.0,1.0,4.0,1.0,1.0,1.0
10,1.0,5.0,1.0,1.0,1.0,1.0,2.0,4.0,1.0,1.0,3.0,1.0,1.0,1.0


In [129]:
rank_df.sum(axis=0), rfe_results

(dist               13.0
 enzyme_class_1    104.0
 enzyme_class_2     28.0
 enzyme_class_3     23.0
 enzyme_class_4     14.0
 enzyme_class_5     16.0
 enzyme_class_6     68.0
 enzyme_class_7     91.0
 n_O                58.0
 n_N                49.0
 n_P                79.0
 n_S                34.0
 n_X                19.0
 DoU                41.0
 dtype: float64,       score
 1  0.926599)

In [126]:

rfe_results = pd.DataFrame(columns=['score']) 

rank_array = np.zeros((17,18))

for i in range(1,17):
    logreg2=linear_model.LogisticRegression(solver='liblinear', penalty='l1', random_state=1, max_iter=1000, class_weight='balanced') 

    rfe = RFE(logreg2, i)
    rfe = rfe.fit(feature_train, np.ravel(reaction_train))
    score = rfe.score(feature_test, reaction_test)

    rfe_results.loc[i, 'score'] = score
    for j in range(0,17):  
        rank_array[i, j] = rfe.ranking_[j]
        

IndexError: index 14 is out of bounds for axis 0 with size 14

In [125]:
rank_df = pd.DataFrame(rank_array, columns=['dist', 'enzyme_class_1', 'enzyme_class_2', 'enzyme_class_3',
       'enzyme_class_4', 'enzyme_class_5', 'enzyme_class_6', 'enzyme_class_7',
       'n_C', 'n_H', 'n_O', 'n_N', 'n_P', 'n_S', 'n_X', 'DoU', 'MW', 'drop'])
#rfe_results
rank_df.drop(rank_df.index[0], inplace=True)
rank_df.drop(['drop'], axis=1, inplace=True)
rank_df

ValueError: Shape of passed values is (15, 14), indices imply (18, 14)

In [100]:
rank_df.sum(axis=0)
# lose MW, n_X, 

dist               16.0
enzyme_class_1    136.0
enzyme_class_2     61.0
enzyme_class_3     31.0
enzyme_class_4     19.0
enzyme_class_5     17.0
enzyme_class_6    121.0
enzyme_class_7    107.0
n_C               152.0
n_H                82.0
n_O                44.0
n_N                52.0
n_P                71.0
n_S                26.0
n_X                22.0
DoU                37.0
MW                 94.0
dtype: float64

In [101]:
rfe_results

,score
1,0.936435
2,0.936435
3,0.937746
4,0.931848
5,0.933159
6,0.931848
7,0.920052
8,0.921363
9,0.918087
10,0.918742


In [ ]:
logit_roc_auc = roc_auc_score(reaction_test, logreg1.predict(feature_test))
fpr, tpr, thresholds = roc_curve(reaction_test, logreg1.predict_proba(feature_test)[:,1])
plt.figure()
plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % logit_roc_auc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.savefig('Log_ROC')
plt.show()

In [39]:
feature_df.columns

Index(['dist', 'enzyme_class_1', 'enzyme_class_2', 'enzyme_class_3',
       'enzyme_class_4', 'enzyme_class_5', 'enzyme_class_6', 'enzyme_class_7',
       'n_C', 'n_H', 'n_O', 'n_N', 'n_P', 'n_S', 'n_X', 'DoU', 'MW'],
      dtype='object')

In [73]:
non_enzyme_feature_df = feature_df.copy()
non_enzyme_feature_df.drop(['enzyme_class_1', 'enzyme_class_2', 'enzyme_class_3',
       'enzyme_class_4', 'enzyme_class_5', 'enzyme_class_6', 'enzyme_class_7'], axis=1, inplace=True)
non_enzyme_feature_df



,dist,n_C,n_H,n_O,n_N,n_P,n_S,n_X,DoU,MW
0,0.421512,5.0,8.0,4.0,2.0,0.0,0.0,0.0,3.0,160.129
1,0.467093,5.0,8.0,4.0,2.0,0.0,0.0,0.0,3.0,160.129
2,0.421512,5.0,6.0,2.0,2.0,0.0,0.0,0.0,4.0,126.115
3,0.414081,5.0,6.0,2.0,2.0,0.0,0.0,0.0,4.0,126.115
4,0.420926,5.0,6.0,2.0,2.0,0.0,0.0,0.0,4.0,126.115
5,0.446031,5.0,6.0,2.0,2.0,0.0,0.0,0.0,4.0,126.115
6,0.421512,4.0,4.0,2.0,2.0,0.0,0.0,0.0,4.0,112.088
7,0.439218,4.0,4.0,2.0,2.0,0.0,0.0,0.0,4.0,112.088
8,0.414081,4.0,4.0,2.0,2.0,0.0,0.0,0.0,4.0,112.088
9,0.420926,4.0,4.0,2.0,2.0,0.0,0.0,0.0,4.0,112.088


In [91]:
features = np.array(non_enzyme_feature_df) #shape balance array for regression
reactions = list(master_df['reacts'])

feature_train, feature_test, reaction_train, reaction_test = train_test_split(features, reactions,
                                                  test_size=0.20, random_state=42)

In [92]:
non_enzyme_rfe_results = pd.DataFrame(columns=['score']) 

non_enzyme_rank_array = np.zeros((11,11))

for i in range(1,11):
    logreg2=linear_model.LogisticRegression(solver='liblinear', penalty='l1', random_state=1, max_iter=1000, class_weight='balanced') 

    rfe = RFE(logreg2, i)
    rfe = rfe.fit(feature_train, np.ravel(reaction_train))
    score = rfe.score(feature_test, reaction_test)

    non_enzyme_rfe_results.loc[i, 'score'] = score
    for j in range(0,10):  
        non_enzyme_rank_array[i, j] = rfe.ranking_[j]
        

In [93]:
non_enzyme_rank_df = pd.DataFrame(non_enzyme_rank_array, columns=['dist',
       'n_C', 'n_H', 'n_O', 'n_N', 'n_P', 'n_S', 'n_X', 'DoU', 'MW', 'drop'])
#rfe_results
non_enzyme_rank_df.drop(non_enzyme_rank_df.index[0], inplace=True)
non_enzyme_rank_df.drop(['drop'], axis=1, inplace=True)
non_enzyme_rank_df

,dist,n_C,n_H,n_O,n_N,n_P,n_S,n_X,DoU,MW
1,1.0,10.0,8.0,6.0,5.0,7.0,3.0,2.0,4.0,9.0
2,1.0,9.0,7.0,5.0,4.0,6.0,2.0,1.0,3.0,8.0
3,1.0,8.0,6.0,4.0,3.0,5.0,1.0,1.0,2.0,7.0
4,1.0,7.0,5.0,3.0,2.0,4.0,1.0,1.0,1.0,6.0
5,1.0,6.0,4.0,2.0,1.0,3.0,1.0,1.0,1.0,5.0
6,1.0,5.0,3.0,1.0,1.0,2.0,1.0,1.0,1.0,4.0
7,1.0,4.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0
8,1.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0
9,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
10,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [94]:
non_enzyme_rank_df.sum(axis=0)
# ditch n_N, MW, n_H, n_P

dist    10.0
n_C     55.0
n_H     38.0
n_O     25.0
n_N     20.0
n_P     31.0
n_S     13.0
n_X     11.0
DoU     16.0
MW      46.0
dtype: float64

In [86]:
features = list(master_df['dist']) #shape balance array for regression
reactions = list(master_df['reacts'])

feature_train, feature_test, reaction_train, reaction_test = train_test_split(features, reactions,
                                                  test_size=0.20, random_state=42)
a = len(feature_train)
b = len(feature_test)
feature_train = np.reshape(feature_train, (a, 1))
feature_test = np.reshape(feature_test, (b,1))

In [89]:

dist_only_model=linear_model.LogisticRegression(solver='liblinear', penalty='l1', random_state=1, max_iter=1000, class_weight='balanced') 

dist_only_model.fit(feature_train, np.ravel(reaction_train))

predictions = dist_only_model.predict(feature_test) # change me to the data you want to predict based on 

score = dist_only_model.score(feature_test, reaction_test)
decision = dist_only_model.decision_function(feature_test)
params = dist_only_model.get_params()
pred_log = dist_only_model.predict_log_proba(feature_test)

pred = dist_only_model.predict_proba(feature_test)

score, pred, decision, dist_only_model.classes_, dist_only_model.coef_
        

(0.936435124508519, array([[0.90331985, 0.09668015],
        [0.64464838, 0.35535162],
        [0.777695  , 0.222305  ],
        ...,
        [0.0098376 , 0.9901624 ],
        [0.51115784, 0.48884216],
        [0.70442001, 0.29557999]]), array([-2.23466854, -0.59559723, -1.25228412, ...,  4.61165718,
        -0.04463876, -0.8684353 ]), array([0., 1.]), array([[17.87916795]]))

kf = KFold(n_splits=10, shuffle=True)
kf.get_n_splits(features, reactions)

int_reactions = [int(i) for i in reactions]


for train_index, test_index in kf.split(features, reactions):
    #print("TRAIN:", train_index, "TEST:", test_index)
    feature_train, feature_test = features[train_index], features[test_index]
    reaction_train, reaction_test = np.array(int_reactions)[train_index], np.array(int_reactions)[test_index]

    reg = linear_model.LogisticRegression().fit(feature_train, reaction_train)
    y_pred = reg.predict(feature_test)

 

print(classification_report(reaction_test, y_pred))

logit_roc_auc = roc_auc_score(reaction_test, reg.predict(feature_test))
fpr, tpr, thresholds = roc_curve(reaction_test, reg.predict_proba(feature_test)[:,1])
plt.figure()
plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % logit_roc_auc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.savefig('Log_ROC')
plt.show()

plt.scatter(reaction_test, y_pred, alpha=0.1)
plt.xlabel('True value')
plt.ylabel('Predicted value')

logreg3 = linear_model.LogisticRegression(solver='sag', max_iter=100000)
logreg3.fit(feature_train, np.ravel(reaction_train))
predictions3 = logreg3.predict(feature_test)

confusion_matrix = confusion_matrix(reaction_test, predictions3)
print(confusion_matrix)

print(classification_report(reaction_test, predictions3))

logit_roc_auc = roc_auc_score(reaction_test, logreg3.predict(feature_test))
fpr, tpr, thresholds = roc_curve(reaction_test, logreg3.predict_proba(feature_test)[:,1])
plt.figure()
plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % logit_roc_auc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.savefig('Log_ROC')
plt.show()

logreg4 = linear_model.LogisticRegression(solver='lbfgs', max_iter=10000)
logreg4.fit(feature_train, np.ravel(reaction_train))
predictions4 = logreg4.predict(feature_test)

logit_roc_auc = roc_auc_score(reaction_test, logreg4.predict(feature_test))
fpr, tpr, thresholds = roc_curve(reaction_test, logreg4.predict_proba(feature_test)[:,1])
plt.figure()
plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % logit_roc_auc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.savefig('Log_ROC')
plt.show()

print(classification_report(reaction_test, predictions4))